# Annotation

# Options & Data

In [1]:
%load_ext autoreload
%autoreload 2

import os
import re
import functools
import numpy as np
import pandas as pd
import corescpy as cr

# Display
pd.options.display.max_colwidth = 1000
pd.options.display.max_columns = 100
pd.options.display.max_rows = 500

# Count Threshold for Cell Quantification
count_threshold = 1

# File Paths
panel = "TUQ97N"
libid = "Stricture-50403C2"
direc = "/mnt/cho_lab/bb-xenium-registry/"
dir_data = os.path.join(direc, f"outputs/{panel}")
dir_writeable = "/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library"
out_dir = os.path.join(dir_writeable, f"outputs/{panel}/nebraska")
mdf = os.path.join(dir_writeable, f"samples_{panel}.csv")  # metadata
path_ann = "~/corescpy/examples/markers_lineages.csv"

# Constants
constants_dict = cr.get_panel_constants(panel_id=panel)
cso, csid, col_condition, col_inflamed, col_stricture, key_stricture = [
    constants_dict[x] if x in constants_dict else None for x in [
        "col_sample_id_o", "col_sample_id", "col_condition", "col_inflamed",
        "col_stricture", "key_stricture"]]  # column & keys in columns

# Clustering Version
c_t = "leiden_res1pt5_dist0_npc30"  # high resolution
# c_t = "leiden_res0pt75_dist0pt3_npc30"  # medium resolution
# c_t = "leiden_res0pt5_dist0pt5_npc30"  # low resolution

# ToppGene Sources & Quantification Count Threshold
srcs = ["Cells of the human intestinal tract mapped across space and time",
        "Human Ileal Epithelial cells from Crohn’s Disease",
        "Human Ileal Immune cells from Crohn’s Disease"]
count_threshold = 1

# Annotation Guide File
anf = pd.read_csv(path_ann)
col_assignment = "group"
assign = anf.dropna(subset=col_assignment).set_index(
    "gene").rename_axis("Gene")  # markers

# Metadata & List of Existing Xenium Data Directories
metadata = (pd.read_excel if mdf[-4:] == "xlsx" else pd.read_csv)(mdf)
if col_stricture is not None and col_condition not in metadata.columns:
    metadata.loc[:, col_condition] = metadata.apply(
        lambda x: key_stricture.capitalize() if x[
            col_stricture].lower() in ["yes", key_stricture.lower()] else x[
                col_inflamed].capitalize(), axis=1)
metadata.loc[:, csid] = metadata[col_condition] + "-" + metadata[cso]
samp_ids = dict(metadata.set_index(cso)[csid])  # map libid to condition-ID
files = functools.reduce(lambda i, j: i + j, [[os.path.join(
    run, i) for i in os.listdir(os.path.join(
        dir_data, run))] for run in os.listdir(dir_data)])

# Data

In [2]:
# Load Spatial Data
file_path = np.array(files)[np.where(["-".join(libid.split(
    "-")[1:]) == os.path.basename(x).split("__")[2].split(
        "-")[0] for x in files])[0][0]]  # find file for sample
self = cr.Spatial(os.path.join(dir_data, file_path), library_id=libid)
self.update_from_h5ad(os.path.join(out_dir, libid + ".h5ad"))
self.get_layer("counts", inplace=True)

# Write Cluster Files
# self.write_clusters(out_dir, col_cell_type=c, overwrite=True,
#                     file_prefix=f"{self._library_id}__", n_top=True)

# Auto-Annotation Results (If Available)
if f"label_{c_t.split('leiden_')[1]}" in self.rna.obs:
    print(f"\n\n{'=' * 80}\n\nAutomatic Annotation Results\n\n{'=' * 80}\n\n")
    self.rna.obs.groupby(c_t).apply(
        lambda x: x[f"label_{c_t.split('leiden_')[1]}"].unique()).apply(
            lambda x: x[0] if len(x) == 1 else x)



<<< INITIALIZING SPATIAL CLASS OBJECT >>>

INFO     reading                                                                                                   
         /mnt/cho_lab/bb-xenium-registry/outputs/TUQ97N/CHO-012/output-XETG00189__0021979__50403C2-TUQ97N-EA__20240
         516__190239/cell_feature_matrix.h5                                                                        




Counts: Initial: table


	Observations: 844288

	Genes: 469







 AnnData object with n_obs × n_vars = 844288 × 469
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'region', 'z_level', 'nucleus_count', 'cell_labels', 'Sample'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs', 'spatial'
    obsm: 'spatial'
    layers: 'counts' 

                gene_ids    feature_types   genome
ABCA7   ENSG00000064687  Gene Expression  Unknown
ACTA2   ENSG00000107796  Gene Expression  Unknown
ACTG2   ENSG00000163017  Gene Expression  Unknown
ADAM15  ENSG00000143537  Gene Expression  Unknown
ADAM28  ENSG00000042980  Gene Expression  Unknown 

  




SpatialData object
├── Images
│     └── 'morphology_focus': DataTree[cyx] (5, 81651, 54057), (5, 40825, 27028), (5, 20412, 13514), (5, 10206, 6757), (5, 5103, 3378)
├── Labels
│    

# ToppGene

In [3]:
if f"label_{c_t.split('leiden_')[1]}" in self.rna.obs:
    print(f"\n\n{'=' * 80}\n\nAutomatic Annotation Results\n\n{'=' * 80}\n\n")
    self.rna.obs.groupby(c_t).apply(
        lambda x: x[f"label_{c_t.split('leiden_')[1]}"].unique()).apply(
            lambda x: x[0] if len(x) == 1 else x)




Automatic Annotation Results





In [18]:
self.get_layer("counts", inplace=True)
tgdf, mks = self.annotate_clusters(
        None, sources=srcs, col_cell_type=c_t, max_results=10000,
        name_pattern={srcs[0]: "SmallIntestine"},
        # p_threshold=1e-15,
        p_threshold=1e-2,
        lfc_threshold=1, n_top_genes=20, n_top_annotations=40)
longs = [(" / Per Region, Age_group, Lineage, cell class, cell type", ""),
         ("SmallIntestine", "SmInt")]
for x in longs:
    tgdf.loc[:, "Name"] = tgdf.Name.apply(lambda y: re.sub(x[0], x[1], y))
tgdf.loc[:, "Name"] = tgdf.Name.apply(lambda y: "|".join(y.split("|")[
        :-1]) if y.split("|")[-1].split("-")[0] in y.split("-")[0] and (any(
            i in y for i in ["Child", "Adult", "Pediatric", "Trim"])) else y)
tgdf = tgdf.rename_axis([c_t, "Rank"])
tgdf = tgdf.join(tgdf.apply(
    lambda x: f"{x['GenesInTermInQuery']} / {x['GenesInQuery']}",
    axis=1).to_frame("Marker Matches"))
tgdf = tgdf[list(tgdf.columns[:1]) + ["Marker Matches"] + list(
        tgdf.columns[1:-1])]
tgdf = tgdf.rename_axis([c_t, "Rank"])
tgdf.sort_index()

Name  \
leiden_res1pt5_dist0_npc30 Rank                                                                                                                                                                 
0                          0                                3'-Pediatric_IBD-SmInt-Hematopoietic-Plasma_cells|Pediatric_IBD / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           1                3'-Pediatric_IBD-SmInt-Hematopoietic-Plasma_cells-IgA_plasma_cell|Pediatric_IBD / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           2                      3'-Child09-12-SmInt-Hematopoietic-Plasma_cells-IgA_plasma_cell|Child09-12 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           3                3'-Pediatric_IBD-SmInt-Hematopoietic-Plasma_cells-IgG_plasma_cell|Pediatric_IBD / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           4                      3'-Child04-06-SmInt-Hematopoietic-Plasma_cells-IgA_plasma_cell|Child04-06 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           5                                      3'-Child09-12-SmInt-Hematopoietic-Plasma_cells|Child09-12 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
1                          0                                      3'-Child09-12-SmInt-Hematopoietic-T_cells-Th17|Child09-12 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           1                                 3'-Pediatric_IBD-SmInt-Hematopoietic-T_cells-Tfh|Pediatric_IBD / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           2                                       3'-Child04-06-SmInt-Hematopoietic-T_cells-Tfh|Child04-06 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
2                          0                   3'-Pediatric_IBD-SmInt-Hematopoietic-Myeloid-LYVE1+_Macrophage|Pediatric_IBD / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           1                       3'-GW_trimst-1-SmInt-Hematopoietic-Myeloid-LYVE1+_Macrophage|GW_trimst-1 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           2                       5'-GW_trimst-2-SmInt-Hematopoietic-Myeloid-LYVE1+_Macrophage|GW_trimst-2 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           3                         3'-Child04-06-SmInt-Hematopoietic-Myeloid-LYVE1+_Macrophage|Child04-06 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           4                       3'-GW_trimst-2-SmInt-Hematopoietic-Myeloid-LYVE1+_Macrophage|GW_trimst-2 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           5                   5'-GW_trimst-1.5-SmInt-Hematopoietic-Myeloid-LYVE1+_Macrophage|GW_trimst-1.5 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           6                         3'-Child09-12-SmInt-Hematopoietic-Myeloid-LYVE1+_Macrophage|Child09-12 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           7                                         3'-GW_trimst-2-SmInt-Hematopoietic-Myeloid|GW_trimst-2 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
3                          0                              3'-Child09-12-SmInt-Mesenchymal-myocytic-myofibroblast|Child09-12 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           1                                            3'-Child09-12-SmInt-Mesenchymal-myocytic|Child09-12 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
                           2                             5'-GW_trimst-2-SmInt-Mesenchymal-myocytic-SMC_(PLPP2+)|GW_trimst-2 / Cellty

# Quantification

## By Markers

In [24]:
%%time

outs, failed = {}, []  # hold results
for i, k in enumerate(self.rna.obs[c_t].unique()):
    if tgdf is not None and k in tgdf.reset_index(1).index.values:
        print(f"{'=' * 80}\n{k}\n ({i + 1}/{len(self.rna.obs[c_t].unique())})"
              f"\n{'=' * 80}\n\n{tgdf.loc[k].iloc[:, :3]}\n\n")
    try:
        outs[k] = self.print_markers(
                k, assign, col_cell_type=c_t, lfc_threshold=2,
                count_threshold=count_threshold, p_threshold=1e-15,
                print_threshold=15, n_top_genes=20)
    except Exception as err:
        failed += [(k, err)]
if len(failed) > 0:
    print(f"Failed: {pd.Series(dict(failed))}")

26
 (1/39)

                                                                                                                                             Name  \
Rank                                                                                                                                                
0                3'-Child09-12-SmInt-Hematopoietic-Myeloid-Mast_cell|Child09-12 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
1          3'-Pediatric_IBD-SmInt-Hematopoietic-Myeloid-Mast_cell|Pediatric_IBD / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
2     5'-GW_trimst-1.5-SmInt-Hematopoietic-Myeloid-CLC+_Mast_cell|GW_trimst-1.5 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   
3                3'-Child04-06-SmInt-Hematopoietic-Myeloid-Mast_cell|Child04-06 / Celltypes from developing, pediatric, Crohn's, & adult GI tract   

     Marker Matches                                                   Genes  
Rank           

## By Specific Genes

In [ ]:
%%time

ggg = pd.DataFrame(["CDKN1A", "CDKN2A", "TP53", "PLAUR", "IL6ST"])[
    0].to_frame("Gene").assign(Annotation="Senescence").set_index("Gene")
outs_bg, failed = {}, []  # hold results
for i, k in enumerate(self.rna.obs[c_t].unique()):
    outs_bg[k] = self.print_markers(
        k, ggg, col_cell_type=c_t, lfc_threshold=None, print_threshold=0,
        count_threshold=count_threshold, p_threshold=None, n_top_genes=None)
if len(failed) > 0:
    print(f"Failed: {pd.Series(dict(failed))}")

## Write

In [6]:
for i, x in enumerate([outs, outs_bg]):
    quant = {}
    for k in outs:
        percs_exp, n_exp = x[k][1].copy(), x[k][2].copy()
        quant[k] = pd.concat([
            x.rename_axis("Measure", axis=1).stack() for x in [
                percs_exp.rename_axis("Cell Types", axis=1).stack().replace(
                    "", np.nan).dropna().to_frame("n_exp"), n_exp.set_index(
                        "Cell Types", append=True).rename({
                            k: "Cluster"}, axis=1)]], keys=[
                                "quantification", "representation"]).unstack(
                                    0).unstack(-1).dropna(how="all", axis=1)
    quant = pd.concat(quant, names=["Cluster"])
    if out_dir:
        suf = "" if i == 0 else "_" + "_".join(ggg.reset_index(
            ).Gene.unique().to_list())  # file suffix if by gene
        q_file = os.path.join(
            out_dir, "quantification",
            f"{self._library_id}__{c_t}_quantification_annotation{suf}.xlsx")
        print(f"Writing to {q_file}")
        quant.to_excel(q_file)

# View

## ToppGene/Quantifications

In [13]:
key_cluster = "34"
output = outs  # see "by markers" version
# output = outs_bg  # see "by specific genes" version

if tgdf is not None and key_cluster in tgdf.reset_index(1).index.values:
    print(f"{'=' * 80}\n\n{key_cluster}\n\n"
          f"{tgdf.loc[key_cluster].iloc[:, :-3]}")
_, percs_exp, n_exp, genes, msg = outs[key_cluster]
n_exp = n_exp.copy().set_index("Cell Types", append=True)
print(percs_exp.applymap(lambda x: x if x == "" else str(
    int(x)) + "%").stack().replace("", np.nan).dropna().sort_values(
            ascending=False).head(20))
print(f"{genes}\n\n{n_exp.applymap(int)}\n\n{msg}")
print(f"\n\nN = {sum(self.rna.obs[c_t] == k)}\n\n")
percs_exp.applymap(lambda x: x if x == "" else str(int(x)) + "%")

Gene                                 
NACA      CLP                            99%
CFB       Mesothelium (PRG4+)            95%
SLC39A8   Mesothelium (PRG4+)            94%
          BEST2+ Goblet cell             94%
COL1A1    Glia 1 (DHH+)                  92%
          Neural                         92%
          Stromal 3 (C7+)                92%
AQP1      Glia 2 (ELN+)                  89%
          Endothelial                    89%
HSP90AA1  ILC1                           88%
SNX3      cDC1                           85%
SORBS2    Adult Glia                     79%
          Contractile pericyte (PLN+)    79%
          Mesenchymal                    79%
          arterial capillary             79%
FOSL2     TRGV4 gdT                      72%
          Monocytes                      72%
DAB2      Myeloid                        71%
          LYVE1+ Macrophage              71%
HSPA1B    myofibroblast (RSPO2+)         64%
dtype: object
group
Adult Glia                              SO

,Adult Glia,BEST2+ Goblet cell,CLP,Contractile pericyte (PLN+),Endothelial,Glia 1 (DHH+),Glia 2 (ELN+),Glia 3 (BCAN+),ILC1,LYVE1+ Macrophage,Mature venous EC,Mesenchymal,Mesothelium (PRG4+),Mesothelium (RGS5+),Monocytes,Myeloid,Neural,Stromal 3 (C7+),TRGV4 gdT,arterial capillary,cDC1,myofibroblast (RSPO2+)
Gene,,,,,,,,,,,,,,,,,,,,,,
AQP1,,,,,89%,,89%,,,,,,,,,,,,,,,
CFB,,,,,,,,,,,,,95%,,,,,,,,,
COL1A1,,,,,,92%,,,,,,,,,,,92%,92%,,,,
DAB2,,,,,,,,,,71%,,,,,,71%,,,,,,
FOSL2,,,,,,,,,,,,,,,72%,,,,72%,,,
HSP90AA1,,,,,,,,,88%,,,,,,,,,,,,,
HSPA1B,,,,,,,,,,,,,,,,,,,,,,64%
NACA,,,99%,,,,,,,,,,,,,,,,,,,
PODXL,,,,,,,,,,,,,,57%,,,,,,,,


## Plotting

## Specific to Subsets of Cell Types

In [ ]:
# comp = ["0", "5", "8", "10", "18", "20", "34"]
comp = ["2", "7", "11", "21", "28"]
count_threshold = 1
key_cluster = "2"
_, percs_exp, n_exp, genes, msg = self.print_markers(
    key_cluster, assign, col_cell_type=c_t, lfc_threshold=None,
    key_compare=comp, count_threshold=count_threshold, p_threshold=1,
    n_top_genes=list(assign[assign.Lump == "Epithelial"].index.intersection(
        self.rna.var_names)) + ["TP53", "PLAUR"])
n_exp = n_exp.copy().set_index("Cell Types", append=True)
print(f"{genes}\n\n{n_exp.applymap(int)}\n\n{msg}")
percs_exp.applymap(lambda x: x if x == "" else str(int(x)) + "%")

# Write All Clusters

In [ ]:
col_fff = "out_file"
fff = np.array(cr.pp.construct_file(directory=direc, panel_id=panel))
bff = np.array([os.path.basename(i) for i in fff])  # base path names
samps = np.array([i.split("__")[2].split("-")[0] for i in fff])
for x in metadata[cso]:
    m_f = metadata[metadata[cso] == x][
        "out_file"].iloc[0]  # ...use to find unconventionally-named files
    locx = np.where(samps == x)[0] if pd.isnull(
        m_f) else np.where(bff == m_f)[0]
    metadata.loc[metadata[cso] == x, col_fff] = fff[locx[0]] if (
        len(locx) > 0) else np.nan  # assign output file to metadata row
metadata = metadata.dropna(subset=[col_fff]).drop_duplicates().set_index(cso)

In [ ]:
# Load Spatial Data
for libid in samp_ids:
    if not os.path.exists(os.path.join(out_dir, samp_ids[libid] + ".h5ad")):
        print(f"\n\nWarning: Critical file(s) for {libid} not found.\n\n")
        continue
    self = cr.Spatial(metadata.loc[libid]["out_file"],
                      library_id=samp_ids[libid])
    self.update_from_h5ad(os.path.join(out_dir, samp_ids[libid] + ".h5ad"))
    if c_t not in self.rna.obs.columns:
        print(f"\n\nWarning: {c_t} column for {libid} not found.\n\n")
        continue
    self.write_clusters(out_dir, col_cell_type=c_t, overwrite=True,
                        file_prefix=f"{self._library_id}__",
                        n_top="find_markers")

In [ ]:
c_t = "leiden_res1pt5_dist0_npc30"
# Faster way for spatial data

# import scanpy as sc

# c_t = "leiden_res1pt5_dist0_npc30"
clusterings = ["res1pt5_dist0_npc30"]

for libid in samp_ids:
    if not os.path.exists(os.path.join(out_dir, samp_ids[libid] + ".h5ad")):
        print(f"\n\nWarning: Critical file(s) for {libid} not found.\n\n")
        continue
    adata = sc.read(os.path.join(out_dir, samp_ids[libid] + ".h5ad"))
    for r in clusterings:
        c_t = f"leiden_{r}"
        if c_t not in adata.obs.columns:
            print(f"\n\nWarning: {c_t} column for {libid} not found.\n\n")
            continue
        fff = os.path.join(out_dir, f"{samp_ids[libid]}__{c_t}.csv")
        adata.obs.set_index("cell_id")[c_t].to_frame("group").to_csv(fff)
        fmr = pd.read_excel(os.path.join(
            out_dir, "annotation_dictionaries/annotations_all.xlsx"),
                            index_col=[0, 1]).iloc[:, :3].astype(str).dropna()
        if f"{samp_ids[libid]}___leiden_{r}_dictionary.xlsx" not in [
                v[0] for v in fmr.index]:
            print(f"{libid} not in annotation dictionary.")
            continue
        else:
            print(f"{libid} annotated.")
        fmr = fmr.loc[f"{samp_ids[libid]}___leiden_{r}_dictionary.xlsx"]
        mans = dict(fmr["annotation"])
        adata.obs.loc[:, f"manual_{r}"] = adata.obs[f"leiden_{r}"].astype(
            int).astype(str).replace(mans)  # Leiden -> manual annotation
        adata.obs.loc[adata.obs[f"manual_{r}"].isnull(
            ), f"manual_{r}"] = adata.obs.loc[adata.obs[
                f"manual_{r}"].isnull(), f"leiden_{r}"].astype(str)
        adata.obs.loc[:, f"manual_{r}"] = adata.obs[
            f"manual_{r}"].astype("category")  # as categorical
        fff = os.path.join(out_dir, f"{samp_ids[libid]}__manual_{r}.csv")
        adata.obs.set_index("cell_id")[f"manual_{r}"].to_frame(
            "group").to_csv(fff)